In [1]:
import os, math, subprocess
import datetime as dt

#data wrangling
import numpy as np
import pandas as pd

#visualization
import seaborn as sns
import matplotlib.pyplot as plt

#excel library
import xlrd

In [2]:
#Loading data
df_transactions = pd.read_excel('KPMG_cleandata.xlsx', sheet_name= 'Transactions')
df_transactions = df_transactions.astype({"online_order":'bool'})
df_customerdemo = pd.read_excel('KPMG_cleandata.xlsx', sheet_name= 'CustomerDemographic')
df_customeraddress = pd.read_excel('KPMG_cleandata.xlsx', sheet_name= 'CustomerAddress')
df_new = pd.read_excel('KPMG_cleandata.xlsx', sheet_name= 'NewCustomerList')
df_transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10


In [3]:
#check customer id in each sheets:
df_customerdemo['customer_id'].drop_duplicates().count()

3998

In [4]:
df_customeraddress['customer_id'].drop_duplicates().count()

3999

In [5]:
df_transactions['customer_id'].drop_duplicates().count()
#There are 3494 customers only buy once

3494

In [6]:
#show duplication in customer_id of transaction sheet
df_transactions[df_transactions.duplicated(subset = 'customer_id', keep = 'last')]
#There are 16506 royal customers

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10
5,6,25,2339,2017-03-08,True,Approved,Giant Bicycles,Road,medium,medium,1538.99,829.65,2006-11-10
6,7,22,1542,2017-04-21,True,Approved,WeareA2B,Standard,medium,medium,60.34,45.26,1993-07-15
7,8,15,2459,2017-07-15,False,Approved,WeareA2B,Standard,medium,medium,1292.84,13.44,2009-04-12
8,9,67,1305,2017-08-10,False,Approved,Solex,Standard,medium,large,1071.23,380.74,1991-08-05
9,10,12,3262,2017-08-30,True,Approved,WeareA2B,Standard,medium,medium,1231.15,161.60,2004-08-17


In [7]:
#leftjoin 3 sheets:
df_leftjoin1 = pd.merge (df_transactions, df_customerdemo, how= 'left', on = 'customer_id')
df_leftjoin1 = pd.merge (df_leftjoin1, df_customeraddress, how= 'left', on = 'customer_id')
df_leftjoin1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 29 columns):
transaction_id                         20000 non-null int64
product_id                             20000 non-null int64
customer_id                            20000 non-null int64
transaction_date                       20000 non-null datetime64[ns]
online_order                           20000 non-null bool
order_status                           20000 non-null object
brand                                  19803 non-null object
product_line                           19803 non-null object
product_class                          19803 non-null object
product_size                           19803 non-null object
list_price                             20000 non-null float64
standard_cost                          19803 non-null float64
product_first_sold_date                19803 non-null datetime64[ns]
first_name                             19989 non-null object
last_name               

In [8]:
#top10 royal customer in transactions sheet
test = df_leftjoin1.groupby(['customer_id','first_name','last_name','address','postcode','state'])['transaction_id'].agg(['count'])
test = test.reset_index()
test.sort_values(by='count',ascending=0).head(10)

,customer_id,first_name,last_name,address,postcode,state,count
1029,1068,Frazer,Searston,4297 Emmet Lane,2170.0,NSW,14
2109,2183,Jillie,Fyndon,8142 Tomscot Drive,4132.0,QLD,14
2394,2476,Hal,Braddon,567 Scott Park,4118.0,QLD,14
2464,2548,Kristien,Robberts,12 Fairfield Parkway,3029.0,VIC,13
361,377,Levy,Bonavia,259 Barnett Crossing,2265.0,NSW,13
3210,3326,Wes,Crotch,18118 Maple Wood Crossing,2763.0,NSW,13
1256,1302,Ericka,Eggers,9 Sugar Junction,4053.0,QLD,13
2665,2755,Karen,La Wille,2241 Pearson Plaza,4503.0,QLD,13
1099,1140,Deana,Rathbourne,8587 Graceland Way,2206.0,NSW,13
2948,3048,Kenon,Messruther,80302 Stang Parkway,3677.0,VIC,13


In [18]:
#Top 10 targeting demographic job, employment
job = df_leftjoin1.groupby(['job_industry_category'])['transaction_id'].agg(['count'])
job = job.reset_index()
job.sort_values(by='count',ascending=0).head(10)

,job_industry_category,count
5,Manufacturing,4014
2,Financial Services,3886
3,Health,3091
7,Retail,1758
6,Property,1297
4,IT,1084
1,Entertainment,698
0,Argiculture,578
8,Telecommunications,354


In [19]:
#top10 product preferences in transactions sheet
test1 = df_transactions.groupby(['brand','product_line','product_class'])['transaction_id'].agg(['count'])
test1 = test1.reset_index()
test1.sort_values(by='count',ascending=0).head(10)

,brand,product_line,product_class,count
20,Solex,Standard,medium,2532
30,WeareA2B,Standard,medium,2346
3,Giant Bicycles,Standard,medium,1769
27,Trek Bicycles,Standard,medium,1254
15,OHM Cycles,Standard,medium,1079
10,Norco Bicycles,Standard,medium,980
2,Giant Bicycles,Standard,high,785
7,Norco Bicycles,Road,medium,732
24,Trek Bicycles,Road,medium,625
31,WeareA2B,Touring,medium,618


In [64]:
df_leftjoin1.tenure.value_counts().idxmax()


7.0

In [66]:
df_leftjoin1.tenure.value_counts().max()

1190

In [70]:
#top10 tenure by sales
test11 = df_leftjoin1.groupby(['tenure'])['transaction_id'].agg(['count'])
test11 = test11.reset_index()
test11.sort_values(by='count',ascending=0).head(10)

,tenure,count
6,7.0,1190
4,5.0,1096
10,11.0,1096
15,16.0,1067
11,12.0,1060
7,8.0,1032
13,14.0,1019
8,9.0,995
9,10.0,985
16,17.0,985


In [76]:
#Target NCL tenure = 7
df_new[df_new.tenure == 7]

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
26,Dyane,Burwell,Female,19,1952-06-27,Engineer II,Retail,Mass Customer,N,Yes,7,51 Hooker Court,2640,NSW,Australia,2,15,1468750
32,Sybilla,MacCart,Female,88,1987-01-15,Paralegal,Financial Services,Mass Customer,N,Yes,7,74 Welch Pass,2620,NSW,Australia,7,17,1453125
64,Cristen,Maroney,Female,22,1999-11-21,Accountant IV,IT,High Net Worth,N,Yes,7,27 Karstens Crossing,4214,QLD,Australia,7,29,1362500
110,Franciska,Stigell,Female,95,1968-11-15,Food Chemist,Health,Mass Customer,N,Yes,7,6 Anderson Junction,3802,VIC,Australia,7,42,1281250
119,Viki,Coutts,Female,2,1952-12-14,Automation Specialist II,Property,Mass Customer,N,Yes,7,6 Golf View Alley,2287,NSW,Australia,4,44,1262500
157,Roseanne,Caruth,Female,16,1957-05-25,Programmer Analyst III,Health,Mass Customer,N,No,7,33652 Lyons Alley,2763,NSW,Australia,9,57,1187500
175,Tanya,Kiefer,Female,54,1992-07-05,Speech Pathologist,Telecommunications,High Net Worth,N,No,7,4 Warner Park,2146,NSW,Australia,9,61,1168750
191,Franklin,Wohler,Male,55,1938-10-16,Clinical Specialist,Health,Mass Customer,N,Yes,7,6966 Delladonna Street,2250,NSW,Australia,8,66,1150000
193,Ermentrude,Heindle,Female,47,1969-08-02,GIS Technical Architect,Argiculture,Mass Customer,N,No,7,307 Knutson Center,2226,NSW,Australia,10,66,1150000
196,Mandie,Jeffryes,Female,60,1981-02-16,Business Systems Development Analyst,NaN,High Net Worth,N,No,7,96515 Di Loreto Pass,4109,QLD,Australia,9,67,1147500


In [11]:
#Lý thuyết JOIN: https://techtalk.vn/loat-series-ve-sql-moi-thu-ban-can-biet-ve-sql-joins.html
#Transactions sheet has 20000 Customer ID, customerdemographic and customeraddress have 4000 Customer ID 
df_rightjoin = pd.merge (df_customerdemo, df_customeraddress, how= 'right', on = 'customer_id')
df_rightjoin.head(3)
#There are only 3774 rows, there are huge missing data => inner join isnt appropriate 

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation
0,3549,Jodee,Macewan,Female,0.0,1966-02-24,Sales Representative,Retail,Mass Customer,N,No,7.0,360 Green Ridge Junction,2508,NSW,Australia,9
1,3646,Allene,Pauly,Female,0.0,1981-10-22,Social Worker,Health,Mass Customer,N,No,4.0,2809 Anderson Lane,3350,VIC,Australia,3
2,3843,Mahmoud,Ligerton,Male,0.0,1989-06-06,Senior Financial Analyst,Financial Services,Mass Customer,N,Yes,16.0,9570 Sundown Trail,4073,QLD,Australia,8


In [12]:
df_leftjoin = pd.merge (df_transactions, df_customeraddress, how= 'left', on = 'customer_id')
df_leftjoin.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,address,postcode,state,country,property_valuation
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02,984 Hoepker Court,3064.0,VIC,Australia,6.0
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03,4 Shopko Circle,2196.0,NSW,Australia,5.0
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20,586 Miller Parkway,2835.0,NSW,Australia,1.0
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16,1617 Harper Parkway,2096.0,NSW,Australia,10.0
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10,601 Stephen Place,2292.0,NSW,Australia,7.0


In [13]:
#find out top current customer location
test2 = df_leftjoin.groupby(['state'])['transaction_id'].agg(['count'])
test2 = test2.reset_index()
test2.sort_values(by='count',ascending=0)

,state,count
0,NSW,10685
2,VIC,5021
1,QLD,4262


In [14]:
#top 10 target new customers
df_new.sort_values(by='Rank', ascending=1).head(10)

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,2,1703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,2,1703125
5,Druci,Brandli,Female,39,1951-04-29,Assistant Media Planner,Entertainment,High Net Worth,N,Yes,22,89105 Pearson Terrace,4075,QLD,Australia,7,3,1671875
6,Rutledge,Hallt,Male,23,1976-10-06,Compensation Analyst,Financial Services,Mass Customer,N,No,8,7 Nevada Crossing,2620,NSW,Australia,7,3,1671875
7,Nancie,Vian,Female,74,1972-12-27,Human Resources Assistant II,Retail,Mass Customer,N,Yes,10,85 Carioca Point,4814,QLD,Australia,5,4,1656250
8,Duff,Karlowicz,Male,50,1972-04-28,Speech Pathologist,Manufacturing,Mass Customer,N,Yes,5,717 West Drive,2200,NSW,Australia,10,4,1656250
9,Barthel,Docket,Male,72,1985-08-02,Accounting Assistant IV,IT,Mass Customer,N,Yes,17,80 Scofield Junction,4151,QLD,Australia,5,5,1640625


In [15]:
#find out top new target customer location
test5 = df_new.groupby(['state'])['first_name'].agg(['count'])
test5 = test5.reset_index()
test5.sort_values(by='count',ascending=0)

,state,count
0,NSW,506
2,VIC,266
1,QLD,228


In [16]:
df_rightjoin = pd.merge (df_customerdemo, df_customeraddress, how= 'right', on = 'customer_id')
df_rightjoin.head() 

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation
0,3549,Jodee,Macewan,Female,0.0,1966-02-24,Sales Representative,Retail,Mass Customer,N,No,7.0,360 Green Ridge Junction,2508,NSW,Australia,9
1,3646,Allene,Pauly,Female,0.0,1981-10-22,Social Worker,Health,Mass Customer,N,No,4.0,2809 Anderson Lane,3350,VIC,Australia,3
2,3843,Mahmoud,Ligerton,Male,0.0,1989-06-06,Senior Financial Analyst,Financial Services,Mass Customer,N,Yes,16.0,9570 Sundown Trail,4073,QLD,Australia,8
3,3976,Gretel,Chrystal,Female,0.0,1957-11-20,Internal Auditor,NaN,Affluent Customer,N,Yes,13.0,7 Morning Court,3805,VIC,Australia,7
4,2920,Casar,Ritchley,U,1.0,NaT,Business Systems Development Analyst,IT,Mass Customer,N,Yes,NaN,7066 Sunbrook Terrace,2299,NSW,Australia,8


In [17]:
#customers from the same address
CA_dup = df_rightjoin[df_rightjoin.duplicated(['address'],keep=False)]
CA_dup

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation
576,2320,Kaylyn,Throssell,Female,14.0,1965-12-21,Senior Developer,Manufacturing,Affluent Customer,N,No,16.0,64 Macpherson Junction,2208,NSW,Australia,11
2049,3540,Ali,Naris,Male,49.0,1954-08-27,NaN,NaN,Mass Customer,N,Yes,7.0,64 Macpherson Junction,4061,QLD,Australia,8
2771,2475,Cammi,Ambrogioni,Female,68.0,1977-05-02,Quality Control Specialist,NaN,Mass Customer,N,No,20.0,3 Talisman Place,4017,QLD,Australia,5
2821,2985,Kerby,Nesfield,Male,69.0,1986-10-14,Sales Representative,Retail,Affluent Customer,N,Yes,14.0,3 Mariners Cove Terrace,2216,NSW,Australia,10
3050,737,Sisely,Oppy,Female,75.0,1983-07-11,Systems Administrator III,NaN,Mass Customer,N,No,16.0,3 Talisman Place,4811,QLD,Australia,2
3347,2333,Ives,Adolfson,Male,82.0,1966-04-19,Structural Analysis Engineer,Financial Services,Mass Customer,N,Yes,10.0,3 Mariners Cove Terrace,3108,VIC,Australia,10
